In [1]:
import os
import glob
import sys
import json
import time
import copy
import torch
import click
import pickle
import numpy as np
import logging as log
import cv2
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import albumentations as A
import torch, torch.nn as nn
import random
import segmentation_models_pytorch as sm

from tqdm import tqdm
from pathlib import Path
from skimage.io import imread
from torch.autograd import Variable
from losses_pytorch.segmentation.twodim.entropy_based_losses import cross_entropy_with_logits_loss

from precode.models import UnetSm
from precode.scoring import jaccard_score
from precode.labels.wheat_segmentation import WHEAT_LABELS
from precode.net_utils import init_determenistic, batch_ids_generator

from skimage.morphology import skeletonize
from skimage import data
import matplotlib.pyplot as plt
from skimage.util import invert


In /opt/tljh/user/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/tljh/user/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/tljh/user/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /opt/tljh/user/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/tljh/user/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mpl

In [2]:
model_path = '/home/jupyter-fly_cv/Igor/Plants/13-45:24-09-21_model_plant_segmentation.bin'
model_path_2 = '/home/jupyter-fly_cv/Igor/Plants/06-44:15-03-22_model_plant_segmentation.bin'
model_path_3 = '/home/jupyter-fly_cv/Igor/Plants/03-10:17-03-22_model_plant_segmentation.bin'
model_path_4 = '/home/jupyter-fly_cv/Igor/Plants/04-39:24-03-22_model_plant_segmentation.bin'
model_path_5 = '/home/jupyter-fly_cv/Igor/Plants/03-23:25-03-22_model_plant_segmentation.bin'
model_path_6 = '/home/jupyter-fly_cv/Igor/Plants/14-47:11-04-22_model_plant_segmentation.bin'
model_path_7 = '/home/jupyter-fly_cv/Igor/Plants/15-40:09-04-22_model_plant_segmentation.bin'
model_checker_path = '/home/jupyter-fly_cv/Igor/Plants/model_segmentation_checker.bin'
test_path = '/home/jupyter-fly_cv/Igor/Plants/Segment_dataset/test'
train_path = '/home/jupyter-fly_cv/Igor/Plants/Segment_dataset/train'
val_path = '/home/jupyter-fly_cv/Igor/Plants/Segment_dataset/val'
img_path = '/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды/'
img_path_pl_2 = '/home/jupyter-fly_cv/Igor/Plants_2/Train_old'
img_path_new = '/home/jupyter-fly_cv/Igor/Plants/data/Новый признак/'
train_path_new = '/home/jupyter-fly_cv/Igor/Plants/New_json/train'
test_path_new = '/home/jupyter-fly_cv/Igor/Plants/New_json/test'

test_pl_2 = '/home/jupyter-fly_cv/Igor/Plants_2/Segment/test'
train_pl_2 = '/home/jupyter-fly_cv/Igor/Plants_2/Segment/train'
val_pl_2 = '/home/jupyter-fly_cv/Igor/Plants_2/Segment/val'

st_checker_size = (870, 1500)
st_img_shape =(4750, 3170)
st_checker_size_new = (200, 350)
st_img_new = (3350, 2950)

In [3]:
def all_jpg_img(path):
    img_names = []

    for dirname, _, filenames in os.walk(path):
        for filename in filenames: 
            name = os.path.join(filename)
            if name.split('.')[-1] != 'JPG':
                continue
            img_names.append(dirname + '/' + name)
            
    return img_names

In [4]:
def concantinate_mask(masks):
    fin_mask = np.zeros((masks[0].shape[0],masks[0].shape[1]))
    for mask in masks:
        fin_mask =fin_mask + mask
    #plt.imshow(fin_mask, cmap = 'gray')
    fin_mask = np.clip(fin_mask,0,1)
    return fin_mask.astype(np.uint8)

In [5]:
a = np.array([np.array([0,1,2.5,366]),
             np.array([0,0,1,0]),
             np.array([0,1,0,0]),
             np.array([0,1,1,0])])
b = np.array([np.array([0,1,2,3]),
             np.array([0,0,1,0]),
             np.array([0,1,0,0]),
             np.array([0,1,1,-1])])
concantinate_mask([a,b])

array([[0, 1, 1, 1],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 1, 1, 0]], dtype=uint8)

In [6]:
def TGI(img,b,r,porog):
    B = img[:,:,0]
    G = img[:,:,1]
    R = img[:,:,2]
    
    TGI = b *B + G + r*R
    
    mask = np.ones((img.shape[0],img.shape[1]))
    mask *= TGI 
    
    mask = cv2.GaussianBlur(mask, (11, 11), cv2.BORDER_DEFAULT)
    mask = cv2.threshold(mask,porog,255,cv2.THRESH_BINARY)[1]
    
    mask = mask.astype(np.uint8)
    
    contours, hierarchy = cv2.findContours(mask,cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    mask_cont = np.zeros((img.shape[0],img.shape[1])).astype(np.uint8)
    cv2.fillPoly(mask_cont,[max_contours(contours)],1)
    mask = mask_cont * mask
    
    mask = np.where(mask > 0, 1, 0).astype(np.uint8)
    
    return mask

In [7]:
def TGI_all(img,b,r,porog):
    B = img[:,:,0]
    G = img[:,:,1]
    R = img[:,:,2]
    
    TGI = b *B + G + r*R
    
    mask = np.ones((img.shape[0],img.shape[1]))
    mask *= TGI 
    
    mask = cv2.GaussianBlur(mask, (11, 11), cv2.BORDER_DEFAULT)
    mask = cv2.threshold(mask,porog,255,cv2.THRESH_BINARY)[1]
    
    mask = mask.astype(np.uint8)

    mask = np.where(mask > 0, 1, 0).astype(np.uint8)
    
    return mask

In [8]:
def get_TGI(img,tgi):
    mask_tgi = TGI(img,tgi[0],tgi[1],tgi[2])
    checker = color_checker_stripe(img,model_checker_path)
    mask_tgi = mask_tgi * checker
    return mask_tgi

In [9]:
def get_TGI_all(img,tgi):
    mask_tgi = TGI_all(img,tgi[0],tgi[1],tgi[2])
    checker = color_checker_stripe(img,model_checker_path)
    mask_tgi = mask_tgi* checker
    return mask_tgi

In [10]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

infer_aug = A.Compose([ 
                        A.Resize ( height=512,
                                   width=512,
                                   interpolation=1,
                                   always_apply=False,
                                   p=1. ),
                        A.Normalize ( mean=(0.485, 0.456, 0.406),
                                      std=(0.229, 0.224, 0.225),
                                      max_pixel_value=255.0,
                                      always_apply=False,
                                      p=1.0 )
                      ])


class UnetSm(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, **kwargs):
        super().__init__()

        self.unet = sm.Unet(in_channels=in_channels, classes=out_channels, **kwargs)

    def forward(self, inputs):
        return self.unet(inputs)

def inverse_infer_aug(height, width):
    return A.Compose([
                       A.Resize ( height=height,
                                  width=width,
                                  interpolation=cv2.INTER_NEAREST,
                                  always_apply=False,
                                  p=1. ) 
                    ])

    
def create_model(model_file):
    model = UnetSm( out_channels=2,
                    encoder_name='efficientnet-b0' )
 
    with open(model_file, 'rb') as f:
        state = pickle.load(f)

    model.load_state_dict(state)
    model.cuda()

    return model


def torch_float(data, device):
    return Variable(torch.FloatTensor(data)).to(device)

def augmented_load(img, aug):
    auged = aug(image=img)
    aug_img = auged['image']

    aug_img = aug_img.transpose(2, 0, 1)

    return np.array([aug_img])

def infer(model, img):
    
    imgs_batch_ = augmented_load(img, infer_aug)
    imgs_batch = torch_float(imgs_batch_, torch.device('cuda'))

    logits_batch = model(imgs_batch)

    pred_masks_batch = logits_batch.argmax(dim=1)
    pred_mask = pred_masks_batch.cpu().data.numpy()#[0]
      #  print(pred_masks_batch.cpu().data.numpy().shape)

    original_size = img.shape[:2]

    original_pred_mask = inverse_infer_aug(*original_size)(image=pred_mask)['image']
    original_pred_mask = original_pred_mask.astype('uint8') * 255 

    return original_pred_mask

In [11]:
def accur(mask_1,mask_2):
    mask_3 = mask_1 * mask_2
    new_mask = np.where(mask_3 > 0, 1,0)
    non_zero_mask = np.nonzero(new_mask)[0]
    if len(non_zero_mask.shape) < 1 or non_zero_mask.shape[0] == 0:
        return 0
    mask_2_new = np.where(mask_2 > 0, 1,0)
    non_zero_mask_2 = np.nonzero(mask_2_new)[0]
    return (non_zero_mask.shape[0])/(non_zero_mask_2.shape[0])

In [12]:
def max_contours(contours):
    
    if len(contours) == 0:
        return np.array([[0,0],[0,1],[1,0]])

    max_cnt = contours[0] 
    max_area_cnt = -1
    
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > max_area_cnt:
            max_area_cnt = area
            max_cnt = cnt
            
    return max_cnt

In [13]:
def filename_pref_png(name, pref):
    names = name.split('/')
    png_names = names[-1].split('.')
    names[-1] = png_names[0] + '.' + 'png'
    names[-1] = pref + names[-1]
    new_name = ''
    for n in names:
        if n != names[-1]:
            new_name =new_name + n + '/'
        else:
            new_name =new_name + n
    return new_name    

In [15]:
#решает задачу о поиске максимума целевой функции func
def Genetic(Q,F,P_mut,number_iter,func):
    
    X_init = list()
    X = list()
    best_f = -99999
    best_x = 'nothing'
    Sample_f = list()
    
    #начальная популяция
    X_init.append( (-0.25,-0.76,0.017) )
   # X_init.append(( -0.4137177782777779,-0.4884255085460936, 21.8252589694986) )
    for i in range(3*Q - 1):
        b = random.uniform(-1,1)
        r = random.uniform(-1,1)
        porog = random.uniform(-20,100)
        X_init.append( (b,r,porog) )
        
    for x in X_init:
        f = func(x[0],x[1],x[2])
        #print(f)
        if f > best_f:
            best_f = f
            best_x = x
            #print(best_f)    
        
        Sample_f.append(f)
        
        #print(-1)
    
    for x in zip(X_init,range(3*Q)):    
        X.append(x)
    
    for i in range(number_iter):
        print(Sample_f)
        
        #размножение(создание пар)  
        Anc = list()
        for x in X:
            p1 = random.choice(X)
            
            X_no_p1 = X.copy()
            X_no_p1.remove(p1)
            
            p2 = random.choice(X_no_p1)
            
            Anc.append( (p1,p2) )
        
        F_pl = 0
        #рождение и отбор
        for par in Anc:
            ch_val = [-1,-1,-1]
            for j in range(3):
                p = random.choice(list(par))
                mut =  random.choices([0,1], weights = [1-P_mut,P_mut])[0]
                mut_2 = random.uniform(-0.001,0.001)
               
                if i >= 50:
                     F_pl =0.5 
                
                if i >= 70:
                     F_pl =1 
                   
                ch_val[j] = p[0][j]*(1 + mut_2)
                if mut == 1:
                    X_no_par = X.copy()
                    if set(par[0]).issubset(X_no_par):
                        X_no_par.remove(par[0])
                    if set(par[1]).issubset(X_no_par):
                        X_no_par.remove(par[1])
                    if len(X_no_par) == 0:
                        an_1 = ((0,0,0),-1)
                        an_2 = ((0,0,0),-1)
                    else:
                        an_1 = random.choice(X_no_par)
                        if set(an_1).issubset(X_no_par):
                            X_no_par.remove(an_1)
                        if len(X_no_par) == 0:
                            an_2 = an_1
                        else:
                            an_2 = random.choice(X_no_par)
                   
                    ch_val[j] += F*(1+F_pl)*(an_1[0][j] - an_2[0][j])
                
            f_ch = func(ch_val[0],ch_val[1],ch_val[2])
            #print(f_ch)
            
            f_1 = Sample_f[par[0][1]]
            f_2 = Sample_f[par[1][1]]
            
            index_bad_par = par[0][1]
            if f_1 > f_2:
                index_bad_par = par[1][1]
        
            if f_ch > Sample_f[index_bad_par]:
                X[index_bad_par] = ((ch_val[0],ch_val[1],ch_val[2]),index_bad_par)
                Sample_f[index_bad_par] = f_ch
                
                if f_ch > best_f:
                    best_f = f_ch
                    best_x = (ch_val[0],ch_val[1],ch_val[2])
                    #print(best_f)
                    
            #print(i)
        
    return best_f,best_x

In [16]:
def strain_contours_accur(contours,mask,porog_accur,porog_a = 0, porog_b = 999999999):
    area_ind = []
    new_cnt = []
    list_area = []
    list_cnt = []
    
    i = 0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > porog_a and area < porog_b:
            area_ind.append((area,i))
            i += 1
            list_cnt.append(cnt)
    
    if len(area_ind) == 0:
        return 0
    
    area_ind = sorted(area_ind, key = lambda x: -x[0])
    
    for i in range(100):
        if i >= len(list_cnt):
            break
        mask_cont = np.zeros((mask.shape[0],mask.shape[1])).astype(np.uint8)
        cnt = list_cnt[area_ind[i][1]]
        cv2.fillPoly(mask_cont,[cnt],1)
#         plt.imshow(mask_cont,cmap = 'gray')
#         print(accur(mask_cont,mask))
#         break
        if accur(mask_cont,mask) > porog_accur :
            new_cnt.append(cnt)
            print('hey')
    
    return np.array(new_cnt)

In [17]:
def rectangle(mask):
    new_mask = np.where(mask > 0, 1,0)
    non_zero_mask = np.nonzero(new_mask)
    h = non_zero_mask[0].max() - non_zero_mask[0].min()
    w = non_zero_mask[1].max() - non_zero_mask[1].min()
    x = set(non_zero_mask[1])
    y = set(non_zero_mask[0])
    x = sum(x)/len(x)
    y = sum(y)/len(y)
    return w,h,x,y

In [18]:
def color_checker_stripe(img,model_checker_path):
    img_checker = infer(create_model(model_checker_path),img)
    img_checker = cv2.GaussianBlur(img_checker, (21, 21), 0)
   # img_checker = np.round(img_checker/255)
   
    img_checker = cv2.threshold(img_checker,15,255,cv2.THRESH_BINARY)[1] 
    
    contours, hierarchy = cv2.findContours(img_checker,cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    img_checker = np.zeros((img.shape[0],img.shape[1])).astype(np.uint8)
    cv2.fillPoly(img_checker,[max_contours(contours)],1)

    checker_stripe = np.zeros((img.shape[0],img.shape[1])).astype(np.uint8)
    w,h,x,y = rectangle(img_checker)
    w_2 = int(w/2) +1
    h_2 = int(h/2) + 1
    x = int(x)
    y = int(y)
    
    im_h = img.shape[0]
    
    points = np.array([(int(x + w_2),int(im_h)),(int(0),int(im_h)),(int(0),int(y - h_2)),(int(x + w_2),int(y - h_2))])
    cv2.fillPoly(checker_stripe,[points],1)   

    return 1 - checker_stripe

In [19]:
def color_checker_rect(img,mask):
    img_checker = infer(create_model(model_checker_path),img)
    img_checker = cv2.GaussianBlur(img_checker, (21, 21), 0)
#    img_checker = np.round(img_checker/255)
   
    img_checker = cv2.threshold(img_checker,19,255,cv2.THRESH_BINARY)[1] 
    
    pot_mask = pot_segment(img,mask)
    img_checker *= 1 - pot_mask
    
    contours, hierarchy = cv2.findContours(img_checker,cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    img_checker = np.zeros((img.shape[0],img.shape[1])).astype(np.uint8)
    cv2.fillPoly(img_checker,[max_contours(contours)],1)

    checker_stripe = np.zeros((img.shape[0],img.shape[1])).astype(np.uint8)
    w,h,x,y = rectangle(img_checker)
    w_2 = int(w/2) +1
    h_2 = int(h/2) + 1
    x = int(x)
    y = int(y)
    
    
    points = np.array([(int(x + w_2),int(y + h_2)),(int(x - w_2),int(y + h_2)),(int(x - w_2),int(y - h_2)),(int(x + w_2),int(y - h_2))])
    cv2.fillPoly(checker_stripe,[points],1)   
    
    return w,h

In [20]:
def plant_rectangle(img,mask,extra_w = 0,extra_h = 0):
    
    w,h,x,y = rectangle(mask)
    #y = img.shape[0] -y
    stripe = np.zeros((img.shape[0],img.shape[1])).astype(np.uint8)
    w *=(1 + extra_w) 
    h *=(1 + extra_h) 
    w_2 = int(w/2)
    h_2 = int(h/2)
    x = int(x)
    
    points = np.array([(int(x + w_2),int(y + h_2)),(int(x - w_2),int(y + h_2)),(int(x - w_2),int(y - h_2)),(int(x + w_2),int(y - h_2))])
    cv2.fillPoly(stripe,[points],1)   
        
    stripe = np.where(stripe > 0, 1,0)
    stripe = stripe.astype(np.uint8)
    
    return stripe

In [21]:
def pot_segment(img,mask):
    img_checker = color_checker_stripe(img,model_checker_path).astype(np.uint8)
    stripe = plant_stripe(img,mask,extra_w = 0.0)
    bl_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) 
    
    new_img = stripe * bl_img*img_checker
    plant_mask = np.where(mask > 0, 0,1).astype(np.uint8)
    new_img *= plant_mask
    new_img = cv2.threshold(new_img,19,255,cv2.THRESH_BINARY)[1]
    
    contours, hierarchy = cv2.findContours(new_img,cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    pot = np.zeros((img.shape[0],img.shape[1])).astype(np.uint8)
    cv2.fillPoly(pot,[max_contours(contours)],1)
    
    plant_mask = np.where(mask > 0, 2,1).astype(np.uint8)  
    w,h,x,y = rectangle(pot)
    pot = np.zeros((img.shape[0],img.shape[1])).astype(np.uint8)
    w = w #* 1.1
    w_2 = int(w/2) +1
    h_2 = int(h/2) + 1
    points = np.array([(int(x + w_2),int(y + h_2)),(int(x - w_2),int(y + h_2)),(int(x - w_2),int(y - h_2)),(int(x + w_2),int(y - h_2))])
    cv2.fillPoly(pot,[points],1)
    
    pot = pot*plant_mask
    pot = np.where(pot == 1, 1,0).astype(np.uint8)
    
    return pot

In [22]:
def plant_stripe(img,mask,extra_w = 0):
    
    w,h,x,y = rectangle(mask)
    stripe = mask.copy()
    w *=(1 + extra_w) 
    w_2 = int(w/2)
    x = int(x) - 1
    
    for i in range(int(w) + 1+100):
        if x-w_2+i >= 0 and  x-w_2+i < img.shape[1]:
            stripe[:,x-w_2+i] = np.zeros(stripe.shape[0]).astype(np.uint8) + 255
        
    stripe = np.where(stripe > 0, 1,0)
    stripe = stripe.astype(np.uint8)
    
    return stripe

In [23]:
def pred_mask(seg_model,model_checker_path, img):
    mask = infer(seg_model,img)
    checker = color_checker_stripe(img,model_checker_path)
    mask = mask * checker
    mask = cv2.GaussianBlur(mask, (11, 11), cv2.BORDER_DEFAULT)
    mask = cv2.threshold(mask,15,255,cv2.THRESH_BINARY)[1]
     
    return mask

In [24]:
def minus_img(img,min_img):
    new_img = img.copy()
    
    if len(img.shape) == 3:
        new_img[:,:,0] = new_img[:,:,0] * min_img
        new_img[:,:,1] = new_img[:,:,1] * min_img
        new_img[:,:,2] = new_img[:,:,2] * min_img
    
    if len(img.shape) == 2:
        new_img = new_img * min_img

    return new_img

In [25]:
def final_segment(img,seg_model, tgi,tgi_all = (-0.7527070259382189, 0.015372226467469639, 18.627924485740596) ):
    
    img_checker = color_checker_stripe(img,model_checker_path).astype(np.uint8)
    mask_unet = pred_mask(seg_model, model_checker_path, img)
    mask_tgi = get_TGI(img,tgi)*img_checker
    mask_tgi_all = get_TGI_all(img,tgi_all)
    mask = concantinate_mask([mask_tgi,mask_unet])
    
    bl_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    
    pot_mask = pot_segment(img,mask)
    plant_mask = plant_rectangle(img,mask)
    
    mask_tgi_all *= (1-pot_mask)
    mask_tgi_all *= plant_mask
    
    return concantinate_mask([mask_tgi,mask_tgi_all])

In [26]:
def augmented_load2(img, aug):
    auged = aug(image=img)
    aug_img = auged['image']

    aug_img = aug_img.transpose(2, 0, 1)

    return np.array([aug_img])

def infer(model, img):
    model.eval()

    with torch.no_grad():
        imgs_batch_ = augmented_load2(img, infer_aug)
        imgs_batch = torch_float(imgs_batch_, config['DEVICE'])

        logits_batch = model(imgs_batch)

        pred_masks_batch = logits_batch.argmax(dim=1)
        pred_mask = pred_masks_batch.cpu().data.numpy()[0]

        original_size = img.shape[:2]

        original_pred_mask = inverse_infer_aug(*original_size)(image=pred_mask)['image']
        original_pred_mask = original_pred_mask.astype('uint8') * 255 

    return original_pred_mask

In [27]:
def load_data():
    data = {}
    datapath = Path(config['DATAPATH'])
    
    for name, folder_name in ([ ('train', 'train'),
                                ('val', 'val') ]):
        imgpathes = list()
        maskpathes = list()
        
        for imgpath in (datapath / folder_name).glob('*.jpg'):
            
            #suff = str(imgpath).split('.')[-1]
            #if suff == 'jpg':
            imgpathes.append(imgpath)
            mask_name = imgpath.name.replace('.png','.png')
            maskpathes.append(imgpath.with_name(mask_name))

        data[name] = (np.array(imgpathes), np.array(maskpathes))
    
    print(data)
    return data

def torch_long(data, device):
    return Variable(torch.LongTensor(data)).to(device)

def torch_float(data, device):
    return Variable(torch.FloatTensor(data)).to(device) #new

In [28]:
def augmented_load(imgpathes, maskpathes, aug):
    images = list()
    masks = list()

    for imgpath, maskpath in zip(imgpathes, maskpathes):
        if os.path.exists(imgpath) and os.path.exists(maskpath):
            img = imread(imgpath)
            ch_stripe = color_checker_stripe(img,model_checker_path)
            img =  minus_img(img,ch_stripe)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
            mask = imread(maskpath,cv2.IMREAD_GRAYSCALE)
            mask = minus_img(mask, ch_stripe)
            mask = mask.clip(max=1)
            
            
            if len(mask.shape) <= 2:
                auged = aug(image=img, mask=mask)
                aug_img = auged['image']
                aug_mask = auged['mask']
        
                aug_img = aug_img.transpose(2, 0, 1)

                images.append(aug_img)
                masks.append(aug_mask)
            
    return np.array(images), np.array(masks)

In [29]:
infer_aug = A.Compose([ 
                        A.Resize ( height=512,
                                   width=512,
                                   interpolation=1,
                                   always_apply=False,
                                   p=1. ),
                        A.Normalize ( mean=(0.485, 0.456, 0.406),
                                      std=(0.229, 0.224, 0.225),
                                      max_pixel_value=255.0,
                                      always_apply=False,
                                      p=1.0 )
                      ])

def inverse_infer_aug(height, width):
    return A.Compose([
                       A.Resize ( height=height,
                                  width=width,
                                  interpolation=cv2.INTER_NEAREST,
                                  always_apply=False,
                                  p=1. ) 
                    ])

In [30]:
def load_test_data(path):
    imgpathes = []
    maskpathes= []
    for imgpath in os.listdir(path):
            if imgpath.split('.')[-1] == 'jpg':
                imgpathes.append(path + '/' + imgpath)
                mask_name = imgpath.replace('.jpg','_mask.png')
                maskpathes.append(path + '/' + mask_name)              
    return imgpathes, maskpathes 

In [31]:
def final_segment_aug(tgi,imgpathes,seg_model, aug):
    masks = list()

    for imgpath in imgpathes:
            
        img = imread(imgpath)
        ch_stripe = color_checker_stripe(img,model_checker_path)
        img =  minus_img(img,ch_stripe)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        mask = final_segment(img,seg_model, tgi)
        mask = minus_img(mask, ch_stripe)
        mask = mask.clip(max=1)
            
            
        auged = aug(image=img, mask=mask)
#            aug_img = auged['image']
        aug_mask = auged['mask']
        
#            aug_img = aug_img.transpose(2, 0, 1)

        masks.append(aug_mask)

            
    return np.array(masks)

In [32]:
def TGI_aug(tgi,imgpathes, aug):
    masks = list()

    for imgpath in imgpathes:
            
            img = imread(imgpath)
            ch_stripe = color_checker_stripe(img,model_checker_path)
            img =  minus_img(img,ch_stripe)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
#            b,r, porog = tgi
        
            mask = get_TGI(img,tgi)
            mask = minus_img(mask, ch_stripe)
            mask = mask.clip(max=1)
            
            auged = aug(image=img, mask=mask)
#            aug_img = auged['image']
            aug_mask = auged['mask']
        
#            aug_img = aug_img.transpose(2, 0, 1)

            masks.append(aug_mask)

            
    return np.array(masks)

In [33]:
def inner_test_loop(model, imgpathes, maskpathes):
    model.eval()
    size = len(maskpathes)

    masks = list()
    pred_masks = list()
    batch_losses = list()

    with torch.no_grad():
            #imgpathes_batch = imgpathes[batch_ids]
            #maskpathes_batch = maskpathes[batch_ids]

            imgs_batch, masks_batch = augmented_load(imgpathes, maskpathes, infer_aug)

            imgs_batch = torch_float(imgs_batch, config['DEVICE']).cpu()      
            masks_batch = torch_long(masks_batch, config['DEVICE']).cpu()

            logits_batch = model(imgs_batch.cuda()).cpu()
            loss = cross_entropy_with_logits_loss(masks_batch, logits_batch)

            pred_masks_batch = logits_batch.argmax(dim=1)
            pred_masks_batch = pred_masks_batch.cpu().data.numpy()

            masks.append(masks_batch)
            #plt.imshow(masks_batch, cmap = 'gray')
            pred_masks.append(pred_masks_batch)
            batch_losses.append(loss.item())

           # del imgpathes_batch, maskpathes_batch, imgs_batch
#     # torch.cuda.empty_cache()

    masks = np.concatenate(masks)
    pred_masks = np.concatenate(pred_masks)
    
    masks = masks.clip(max=1)
    pred_masks = pred_masks.clip(max=1)
    
   #print(type(masks))

    jaccard_score_none = jaccard_score(masks, pred_masks, average='none')

    score = jaccard_score_none[1]

    return score, np.mean(batch_losses)


In [34]:
def jaccard_final_segment(model,tgi,imgpathes, maskpathes):
   
    size = len(maskpathes)

    masks = list()
    pred_masks = list()
    
    with torch.no_grad():
            
            imgs_batch, masks_batch = augmented_load(imgpathes, maskpathes, infer_aug)

            imgs_batch = torch_float(imgs_batch, config['DEVICE']).cpu()      
            masks_batch = torch_long(masks_batch, config['DEVICE']).cpu()
                
            pred_masks_batch = final_segment_aug(tgi,imgpathes, model,infer_aug)
            pred_masks_batch = torch_long(pred_masks_batch, config['DEVICE']).cpu()
            #pred_masks_batch_tgi = pred_masks_batch_tgi.argmax(dim=1)
            #pred_masks_batch = pred_masks_batch.cpu().data.numpy()
            
            masks.append(masks_batch)
            pred_masks.append(pred_masks_batch)

    masks = np.concatenate(masks)
    pred_masks = np.concatenate(pred_masks)
    
    masks = masks.clip(max=1)
    pred_masks = pred_masks.clip(max=1)

    jaccard_score_none = jaccard_score(masks, pred_masks, average='none')

    score = jaccard_score_none[1]

    return score

In [35]:
def jaccard_TGI(tgi,imgpathes, maskpathes):
 
    size = len(maskpathes)

    masks = list()
    pred_masks = list()
    
    with torch.no_grad():
            
            imgs_batch, masks_batch = augmented_load(imgpathes, maskpathes, infer_aug)

            imgs_batch = torch_float(imgs_batch, config['DEVICE']).cpu()      
            masks_batch = torch_long(masks_batch, config['DEVICE']).cpu()

            pred_masks_batch = TGI_aug(tgi,imgpathes,infer_aug)
            pred_masks_batch = torch_long(pred_masks_batch, config['DEVICE']).cpu()
                    
            masks.append(masks_batch)
            pred_masks.append(pred_masks_batch)

    masks = np.concatenate(masks)
    pred_masks = np.concatenate(pred_masks)
    
    masks = masks.clip(max=1)
    pred_masks = pred_masks.clip(max=1)
    
    jaccard_score_none = jaccard_score(masks, pred_masks, average='none')

    score = jaccard_score_none[1]

    return score

In [36]:
def jaccard_multi_model_TGI(models,tgi,imgpathes, maskpathes):
    
    for model in models:
        model.eval()
   
    size = len(maskpathes)
    pred_masks = np.array(list()).astype(np.uint8)
    pred_masks_tgi =  np.array(list()).astype(np.uint8)
    masks = list()
    model_mask = list()
    
    with torch.no_grad():
            
            imgs_batch, masks_batch = augmented_load(imgpathes, maskpathes, infer_aug)

            imgs_batch = torch_float(imgs_batch, config['DEVICE']).cpu()      
            masks_batch = torch_long(masks_batch, config['DEVICE']).cpu()

            for model in models:
                logits_batch = model(imgs_batch.cuda()).cpu()

                pred_masks_batch = logits_batch.argmax(dim=1)
                pred_masks_batch = pred_masks_batch.cpu().data.numpy()
                model_mask.append(pred_masks_batch)
                
            #model_mask.append(pred_masks_batch)
                
            pred_masks_batch_tgi = TGI_aug(tgi,imgpathes, infer_aug)
            pred_masks_batch_tgi = torch_long(pred_masks_batch_tgi, config['DEVICE']).cpu()
           # pred_masks_batch_tgi = pred_masks_batch_tgi.argmax(dim=1)
            #pred_masks_batch_tgi = pred_masks_batch_tgi.cpu().data.numpy()
            #model_mask.append(pred_masks_batch_tgi)
            
            masks.append(masks_batch)

    masks = np.concatenate(masks)
    
    pred_masks = np.concatenate(model_mask[0])
    for m_mask in model_mask:
        pred_masks = concantinate_mask([np.concatenate(m_mask),pred_masks])
    
    pred_masks_tgi = np.concatenate([pred_masks_batch_tgi])
    pred_masks_tgi = np.resize(pred_masks_tgi,(pred_masks.shape[0],pred_masks.shape[1]))
    pred_masks = concantinate_mask([pred_masks,pred_masks_tgi])
    
    pred_masks = np.resize(pred_masks,(masks.shape[0],masks.shape[1],masks.shape[2]))
    
    masks = masks.clip(max=1)
    pred_masks = pred_masks.clip(max=1)
    
    jaccard_score_none = jaccard_score(masks, pred_masks, average='none')

    score = jaccard_score_none[1]

    return score

In [37]:
def jaccard_multi_model(models,imgpathes, maskpathes):
    
    for model in models:
        model.eval()
   
    size = len(maskpathes)
    pred_masks = np.array(list()).astype(np.uint8)
    masks = list()
    model_mask = list()
    
    with torch.no_grad():
            
            imgs_batch, masks_batch = augmented_load(imgpathes, maskpathes, infer_aug)

            imgs_batch = torch_float(imgs_batch, config['DEVICE']).cpu()      
            masks_batch = torch_long(masks_batch, config['DEVICE']).cpu()

            for model in models:
                logits_batch = model(imgs_batch.cuda()).cpu()

                pred_masks_batch = logits_batch.argmax(dim=1)
                pred_masks_batch = pred_masks_batch.cpu().data.numpy()
                model_mask.append(pred_masks_batch)
            
            masks.append(masks_batch)

    masks = np.concatenate(masks)
    
    pred_masks = np.concatenate(model_mask[0])

    for m_mask in model_mask:
        pred_masks = concantinate_mask([np.concatenate(m_mask),pred_masks])
    
    pred_masks = np.resize(pred_masks,(masks.shape[0],masks.shape[1],masks.shape[2]))
  
    masks = masks.clip(max=1)
    pred_masks = pred_masks.clip(max=1)
    
    jaccard_score_none = jaccard_score(masks, pred_masks, average='none')

    score = jaccard_score_none[1]

    return score

In [38]:
model1 = create_model(model_path)
model2 = create_model(model_path_2)
model4 = create_model(model_path_4)
model5 = create_model(model_path_5)

model_pl_2 = create_model(model_path_7)
model_pl_2_dip = create_model(model_path_6)

img_names,mask_names = load_test_data(train_pl_2) 
img_names

print(inner_test_loop(model_pl_2, img_names, mask_names))
print(inner_test_loop(model_pl_2_dip, img_names, mask_names))
#print(inner_test_loop(model3, img_names, mask_names))

#print(jaccard_multi_model([model2],img_names,mask_names))
#print(jaccard_multi_model([model1,model2,model3],img_names,mask_names))
#print(jaccard_multi_model([model1,model2],img_names,mask_names))
#print(jaccard_multi_model([model1,model3],img_names,mask_names))
#print(jaccard_multi_model([model2,model3],img_names,mask_names))


ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
model2 = create_model(model_path_2)
img_names,mask_names = load_test_data(val_path)
print(inner_test_loop(model2, img_names, mask_names))

In [ ]:
pred_masks = np.array(list()).astype(np.uint8)
pred_masks.shape[0]

In [ ]:
img_names,mask_names = load_test_data(test_path_new) 
print(jaccard_final_segment(model2,(-0.4525162720429479, -0.4636879441900241, 20.541210574133643) ,img_names, mask_names))

In [ ]:
#для проверки генетического алгоритма
def f1(a,b,c):
    return -abs(a*a*a+2*b*b+3*c - 500)

In [ ]:
def jaccard_func_tgi(b,r,porog):
    img_names,mask_names = load_test_data(val_path)
    jaccard = jaccard_TGI((b,r,porog),img_names, mask_names)
    return  jaccard

In [ ]:
def jaccard_func_tgi_train_pl2(b,r,porog):
    img_names,mask_names = load_test_data(img_path_pl_2)
    jaccard = jaccard_TGI((b,r,porog),img_names, mask_names)
    return  jaccard

In [ ]:
def jaccard_func_tgi_train(b,r,porog):
    img_names,mask_names = load_test_data(train_path)
    jaccard = jaccard_TGI((b,r,porog),img_names, mask_names)
    return  jaccard

In [ ]:
def jaccard_func_tgi_train_new(b,r,porog):
    img_names,mask_names = load_test_data(train_path_new)
    jaccard = jaccard_TGI((b,r,porog),img_names, mask_names)
    return  jaccard

In [ ]:
jaccard_func_tgi_train(  -0.3888966243829097,-0.5302251579522758, 19.5706352334385)

In [ ]:
jaccard_func_tgi_train(b,r,porog)

In [ ]:
Genetic(5,0.5,0.5,100,jaccard_func_tgi_train_pl2)

In [ ]:
img_names,mask_names = load_test_data(img_path_pl_2)
mask_names

In [ ]:
jaccard_func_tgi( *(-0.428685910442829, -0.49736106371992633, 17.1750795654293))

In [ ]:
Genetic(5,0.5,0.5,10000,f1)

In [ ]:
Genetic(1,1,0.5,10,jaccard_func_tgi)

In [ ]:
best_par = (0,0)
best_jaccard = -1
for F in range(3,6,1):
    for P_mut in range(3,11):
        f = F/10
        p_mut = P_mut/10
        print()
        print(f)
        print(p_mut)
        jaccard,gti = Genetic(2,f,p_mut,10,jaccard_func_tgi)
        print('jacc: ' + str(jaccard))
        if jaccard > best_jaccard:
            best_jaccard = jaccard
            best_par = (f,p_mut)
            print('new best:')
            print(gti)
            print(jaccard)
        
    

In [ ]:
Genetic(3,0.5,0.5,100,jaccard_func_tgi_train)

In [ ]:
model2 = create_model(model_path_2)

In [ ]:
img_names,mask_names = load_test_data(test_path_new)
jaccard_TGI( (-0.5508976263096602, -0.12285704107229999, 21.31430587433073)  ,img_names, mask_names)

In [ ]:
img_names,mask_names = load_test_data(train_path)
jaccard_final_segment(model2,( -0.4636879441900241,-0.4525162720429479, 20.541210574133643)  ,img_names, mask_names)

In [ ]:
img = imread('/home/jupyter-fly_cv/Igor/Plants_2/train_Demo/DJI_0268_mask.png',cv2.IMREAD_GRAYSCALE)[:,:,0]
plt.imshow(img,cmap = 'gray')
img.shape

In [ ]:
img_names,mask_names = load_test_data(test_path_new) 
jaccard_multi_model_TGI([model2],(-0.5508976263096602, -0.12285704107229999, 21.31430587433073),img_names, mask_names)

In [ ]:
# tgi = (-0.428685910442829, -0.49736106371992633, 17.1750795654293)
# #jaccard_train = 0.773232962494171
# #jaccard_val = 0.7796997163218709
# #jaccard_test = 0.8213170879065552
# #jaccard_multi = 0.7471617991160413

In [ ]:
#tgi = (-0.4137177782777779, -0.4884255085460936, 21.8252589694986)
# #jaccard_train = 0.7766485843118405
# #jaccard_val = 0.0.7803111780311178
# #jaccard_test = 0.8213170879065552
# #jaccard_multi = 0.7471617991160413

In [ ]:
#tgi =(-0.4525162720429479, -0.4636879441900241, 20.541210574133643) 
tgi =(-0.5508976263096602,-0.12285704107229999 , 21.31430587433073)

In [ ]:
m_1 = np.array([[0,0,1,0,1],
                [1,0,1,0,1]])
m_2 = np.array([[0,0,1,0,0],
                [1,0,1,5,1]])
accur(m_1,m_2)

In [39]:
def direct_filename(name):
    names = name.split('/')
    direct = ''
    for n in names:
        if n == names[-1]:
            continue
        direct = direct + n + '/' 
    return direct   

In [ ]:
def name_4_folder(path):
    names = path.split('/')
    for name in names:
        if name == '4':
            return True
    return False

In [41]:
seg_model = create_model(model_path)
img_names = all_jpg_img(img_path_new)
checker_model = create_model(model_checker_path)
os.chdir('/home/jupyter-fly_cv/Igor/Plants')

for name in img_names:

    print(name)
    img = cv2.imread(name)
    img_res = cv2.resize(img,(int(st_img_new[1]), int(st_img_new[0])), interpolation = cv2.INTER_AREA)
    print(color_checker_stripe(img,model_checker_path).astype(np.uint8).shape)
    bl_img = cv2.cvtColor(img_res,cv2.COLOR_BGR2GRAY)
    mask_unet = pred_mask(seg_model, model_checker_path, img_res)

    mask_f = final_segment(img_res,seg_model, tgi)*255

    w,h = color_checker_rect(img_res,mask_f);
    print(w,h)
    w = st_checker_size_new[0]/w
    h = st_checker_size_new[1]/h
    
    mask = cv2.resize(mask_f,(int(mask_f.shape[1]*w), int(mask_f.shape[0]*h)), interpolation = cv2.INTER_AREA)
   
    cv2.imwrite('mask_1.png',mask_unet)
    break
    os.chdir(direct_filename(name))
    cv2.imwrite(filename_pref_png(name.split('/')[-1].split('.')[0], 'mask_'), mask)

/home/jupyter-fly_cv/Igor/Plants/data/Новый признак/2022_03_11 Wheat/L-25/IMG_8319.JPG
(4480, 6720)


NameError: name 'tgi' is not defined

In [ ]:
img_names = all_jpg_img(img_path_pl_2)
os.chdir(img_path_pl_2)

for name in all_jpg_img(img_path_pl_2):
    print(name)
    img = cv2.imread(name)
    mask_tgi = get_TGI(img,( -0.8059495495534817,-0.1761912501892932, 15.560133069974423))
    cv2.imwrite(filename_pref_png(name.split('/')[-1].split('.')[0], 'mask_tgi_'), mask_tgi*255)